<a href="https://colab.research.google.com/github/AlexRaudvee/MultiArchPDD-CV/blob/main/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import shutil
import zipfile
from pathlib import Path
from google.colab import drive

def mount_google_drive(mount_point: Path = Path('/content/drive')) -> Path:
    """Mounts Google Drive and returns the mount point."""
    drive.mount(str(mount_point))
    return mount_point

def extract_zip(zip_path: Path, extract_to: Path) -> None:
    """Extracts a zip file to the given directory."""
    if not zip_path.is_file():
        raise FileNotFoundError(f"Could not find zip file at {zip_path}")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(str(extract_to))

def move_contents(src_dir: Path, dst_dir: Path) -> None:
    """
    Moves everything from src_dir into dst_dir.
    Overwrites any existing files or folders of the same name.
    Cleans up the now-empty src_dir at the end.
    """
    if not src_dir.is_dir():
        raise FileNotFoundError(f"{src_dir} does not exist")
    for item in src_dir.iterdir():
        target = dst_dir / item.name
        if target.exists():
            print(f"Warning: {target} already exists, overwriting")
            if target.is_dir():
                shutil.rmtree(target)
            else:
                target.unlink()
        shutil.move(str(item), str(target))
    src_dir.rmdir()

def setup_directories(*dirs: Path) -> None:
    """Ensures that each directory in `dirs` exists."""
    for d in dirs:
        d.mkdir(parents=True, exist_ok=True)

def zip_folder(folder_path: Path, output_path: Path) -> None:
    """
    Recursively zip the contents of folder_path into a .zip file at output_path.
    """
    with zipfile.ZipFile(output_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for fname in files:
                fpath = Path(root) / fname
                arcname = fpath.relative_to(folder_path)
                zipf.write(str(fpath), arcname)

Mounted at /content/drive


In [ ]:
# ——— Constants ———
DRIVE_MOUNT_POINT = Path('/content/drive')
ZIP_PATH            = DRIVE_MOUNT_POINT / 'MyDrive/.colab.zip'
EXTRACT_TO          = Path('/content')
SRC_DIR             = EXTRACT_TO / '.colab'
DST_DIR             = EXTRACT_TO
DISTILLED_DIR       = EXTRACT_TO / 'data' / 'Distilled'
MODEL_DIR           = EXTRACT_TO / 'data' / 'checkpoints'
ASSETS_DIR          = EXTRACT_TO / 'assets' / 'viz_synthetic'

# ——— SetUp ———
mount_google_drive(DRIVE_MOUNT_POINT)
extract_zip(ZIP_PATH, EXTRACT_TO)
move_contents(SRC_DIR, DST_DIR)
setup_directories(DISTILLED_DIR)
setup_directories(ASSETS_DIR)
setup_directories(MODEL_DIR)

In [ ]:
!pip install matplotlib

In [2]:
!python -m scripts.run_distill \
  --pdd-core mm-match \
  --dataset cifar10 \
  --model convnet \
  --batch-size 128 \
  --synthetic-size 10 \
  --P 1 \
  --K 1 \
  --T 1 \
  --lr-model 1e-4 \
  --lr-syn-data 1e-3 \
  --syn-optimizer adam \
  --inner-optimizer momentum \
  --out-dir data/Distilled \
  --ckpt-dir data/checkpoints

[Dataloader]:
     - Loading...
     - Done.
     - Done.                                                                    
[Distillator]:
     - Saving...
[Distillator]:
     - model saved to data/checkpoints/mm-match_cifar10_convnet.pth
[Distillator]:
     - distilled dataset & history saved to data/Distilled/mm-match_cifar10_convnet.pt
     - Done.
     - Plotted & saved stage 1 → assets/viz_synthetic/synthetic_stage_01.png


In [3]:
zip_folder("/content/assets", "/content/assets_ConvNet.zip")
zip_folder("/content/data/Distilled", "/content/Distilled.zip")
zip_folder("/content/data/checkpoints", "/content/checkpoints.zip")